In [70]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import date


In [71]:
ticker = 'TSLA'

In [72]:
urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials'
urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials/balance-sheet'


In [169]:
urlbalancesheet

'https://www.marketwatch.com/investing/stock/TSLA/financials/balance-sheet'

In [154]:
text_soup_financials = BeautifulSoup(requests.get(urlfinancials).text,"html")

In [155]:
text_soup_balancesheet = BeautifulSoup(requests.get(urlbalancesheet).text,"html")

In [156]:
text_soup_financials

<!DOCTYPE html>
<html class="">
<head>
<title>TSLA | Tesla Inc. Annual Income Statement | MarketWatch</title>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="user-scalable=no, initial-scale=1.0, maximum-scale=1.0, width=device-width" name="viewport"/>
<meta content="noarchive, noodp" name="robots"/>
<link href="//sts.wsj.net" rel="dns-prefetch"/>
<link href="//s.marketwatch.com" rel="dns-prefetch"/>
<link href="//video-api.wsj.com" rel="dns-prefetch"/>
<link href="//fonts.wsj.net" rel="dns-prefetch"/>
<link href="//m.wsj.net" rel="dns-prefetch"/>
<link href="//mwstream.wsj.net" rel="dns-prefetch"/>
<link href="//tags.tiqcdn.com" rel="dns-prefetch"/>
<link href="//s.ntv.io" rel="dns-prefetch"/>
<link href="//cdn.cxense.com" rel="dns-prefetch"/>
<link href="//a248.e.akamai.net" rel="dns-prefetch"/>
<link href="//om.dowjoneson.com" rel="dns-prefetch"/>
<link href="//bam.nr-data.net" rel="dns-prefetch"/>
<link href="//www.googletagservices.com"

In [160]:
titlesfinancials = text_soup_financials.findAll('td', class_ = 'overflow__cell fixed--column')
epslist=[]
netincomelist = []
longtermdebtlist = [] 
interestexpenselist = []
ebitdalist= []

In [161]:
titlesfinancials

[<td class="overflow__cell fixed--column">
 <div class="cell__content fixed--cell">Sales/Revenue</div>
 <div class="cell__content">Sales/Revenue</div>
 </td>,
 <td class="overflow__cell fixed--column">
 <div class="cell__content fixed--cell indent--small">Sales Growth</div>
 <div class="cell__content indent--small">Sales Growth</div>
 </td>,
 <td class="overflow__cell fixed--column">
 <div class="cell__content fixed--cell">Cost of Goods Sold (COGS) incl. D&amp;A</div>
 <div class="cell__content">Cost of Goods Sold (COGS) incl. D&amp;A</div>
 </td>,
 <td class="overflow__cell fixed--column">
 <div class="cell__content fixed--cell indent--small">COGS Growth</div>
 <div class="cell__content indent--small">COGS Growth</div>
 </td>,
 <td class="overflow__cell fixed--column">
 <div class="cell__content fixed--cell indent--small">COGS excluding D&amp;A</div>
 <div class="cell__content indent--small">COGS excluding D&amp;A</div>
 </td>,
 <td class="overflow__cell fixed--column">
 <div class="c

In [164]:
for title in titlesfinancials:
        if 'EPS (Basic)' in title.text:
            epslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'overflow__cell'}) if td.text])
        if 'Net Income' in title.text:
            netincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'overflow__cell'}) if td.text])
        if 'Interest Expense' in title.text:
            interestexpenselist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'overflow__cell'}) if td.text])
        if 'EBITDA' in title.text:
            ebitdalist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'overflow__cell'}) if td.text])

In [171]:
    # find the table headers for the Balance sheet
    titlesbalancesheet = text_soup_balancesheet.findAll('td', {'class': 'overflow__cell fixed--column'})
    equitylist=[]

In [172]:
titlesbalancesheet

[<td class="overflow__cell fixed--column">
 <div class="cell__content fixed--cell">Cash &amp; Short Term Investments</div>
 <div class="cell__content">Cash &amp; Short Term Investments</div>
 </td>,
 <td class="overflow__cell fixed--column">
 <div class="cell__content fixed--cell indent--small">Cash &amp; Short Term Investments Growth</div>
 <div class="cell__content indent--small">Cash &amp; Short Term Investments Growth</div>
 </td>,
 <td class="overflow__cell fixed--column">
 <div class="cell__content fixed--cell indent--small">Cash Only</div>
 <div class="cell__content indent--small">Cash Only</div>
 </td>,
 <td class="overflow__cell fixed--column">
 <div class="cell__content fixed--cell indent--small">Short-Term Investments</div>
 <div class="cell__content indent--small">Short-Term Investments</div>
 </td>,
 <td class="overflow__cell fixed--column">
 <div class="cell__content fixed--cell indent--small">Cash &amp; ST Investments / Total Assets</div>
 <div class="cell__content inden

In [173]:
for title in titlesbalancesheet:
        if 'Total Shareholders\' Equity' in title.text:
            equitylist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'overflow__cell'}) if td.text])
        if 'Long-Term Debt' in title.text:
            longtermdebtlist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'overflow__cell'}) if td.text])

In [175]:
longtermdebtlist

[['2.27B', '7.39B', '11.15B', '11.12B', '12.63B', ' '],
 ['2.07B', '5.9B', '8.83B', '8.41B', '10.4B', ' ']]

In [180]:
    #get the data from the income statement lists 
    #use helper function get_element
    eps = get_element(epslist,0)
    epsGrowth = get_element(epslist,1)
    netIncome = get_element(netincomelist,0)
    shareholderEquity = get_element(equitylist,0)
    roa = get_element(equitylist,1)

    longtermDebt = get_element(longtermdebtlist,0)
    interestExpense =  get_element(interestexpenselist,0)
    ebitda = get_element(ebitdalist,0)

In [185]:
fin_df= pd.DataFrame({'eps': eps,'eps Growth': epsGrowth,'net Income': netIncome,'shareholder Equity': shareholderEquity,'roa': 
                  roa,'longterm Debt': longtermDebt,'interest Expense': interestExpense,'ebitda': ebitda},index=range(date.today().year-6,date.today().year))
    
fin_df.reset_index(inplace=True)

In [179]:
def get_element(list,element):
    try:
        return list[element]
    except:
        return '-'

In [186]:
fin_df

,index,eps,eps Growth,net Income,shareholder Equity,roa,longterm Debt,interest Expense,ebitda
0,2015,(1.39),-,(888.66M),1.08B,13.43%,2.27B,118.85M,(294.04M)
1,2016,(0.94),32.47%,(773.05M),4.75B,20.97%,7.39B,198.81M,256.56M
2,2017,(2.37),-152.78%,(2.24B),4.24B,14.79%,11.15B,471.26M,(1.68M)
3,2018,(1.14),51.61%,(1.06B),4.92B,16.55%,11.12B,663.07M,1.65B
4,2019,(0.97),14.92%,(775M),6.62B,19.29%,12.63B,685M,2.23B
5,2020,,,,,,,,


In [187]:
epslist[0]

['(1.39)', '(0.94)', '(2.37)', '(1.14)', '(0.97)', ' ']

In [188]:
fin_2 = fin_df.drop(axis=0, index=5)

In [189]:
fin_2

,index,eps,eps Growth,net Income,shareholder Equity,roa,longterm Debt,interest Expense,ebitda
0,2015,(1.39),-,(888.66M),1.08B,13.43%,2.27B,118.85M,(294.04M)
1,2016,(0.94),32.47%,(773.05M),4.75B,20.97%,7.39B,198.81M,256.56M
2,2017,(2.37),-152.78%,(2.24B),4.24B,14.79%,11.15B,471.26M,(1.68M)
3,2018,(1.14),51.61%,(1.06B),4.92B,16.55%,11.12B,663.07M,1.65B
4,2019,(0.97),14.92%,(775M),6.62B,19.29%,12.63B,685M,2.23B


# Stack Overflow example

In [107]:
url = "https://www.marketwatch.com/investing/stock/khc/profile"
html_content = requests.get(url).text

In [147]:
soup = BeautifulSoup(html_content, "lxml")
sections = soup.findAll('div', attrs={'class' : 'element element--table'})

In [148]:
sections

[<div class="element element--table">
 <header class="header header--secondary">
 <h2 class="title">
 <span class="label">VALUATION</span>
 </h2>
 </header>
 <table class="table value-pairs no-heading">
 <tbody>
 <tr class="table__row">
 <td class="table__cell w75">P/E Current</td>
 <td class="table__cell w25">20.74</td>
 </tr>
 <tr class="table__row">
 <td class="table__cell w75">P/E Ratio (w/ extraordinary items)</td>
 <td class="table__cell w25 is-na">N/A</td>
 </tr>
 <tr class="table__row">
 <td class="table__cell w75">P/E Ratio (w/o extraordinary items)</td>
 <td class="table__cell w25">20.00</td>
 </tr>
 <tr class="table__row">
 <td class="table__cell w75">Price to Sales Ratio</td>
 <td class="table__cell w25">1.55</td>
 </tr>
 <tr class="table__row">
 <td class="table__cell w75">Price to Book Ratio</td>
 <td class="table__cell w25">0.75</td>
 </tr>
 <tr class="table__row">
 <td class="table__cell w75">Price to Cash Flow Ratio</td>
 <td class="table__cell w25">10.90</td>
 </tr>
 

In [131]:
for section in sections : 
    ps = section.find('td')
    if "P/E Current" in ps.getText() or "Price to Sales Ratio" in ps.getText(): 
        val = ps.nextSibling.nextSibling
        print(f"{ps.getText()}: {val.getText()}")

P/E Current: 20.74


In [151]:
all_data=[]
for item in sections:
    data ={}
    print(item)
    data['name']= item.tr.text
    data['value']=item.tr.findNext('td').text
    all_data.append(data)

<div class="element element--table">
<header class="header header--secondary">
<h2 class="title">
<span class="label">VALUATION</span>
</h2>
</header>
<table class="table value-pairs no-heading">
<tbody>
<tr class="table__row">
<td class="table__cell w75">P/E Current</td>
<td class="table__cell w25">20.74</td>
</tr>
<tr class="table__row">
<td class="table__cell w75">P/E Ratio (w/ extraordinary items)</td>
<td class="table__cell w25 is-na">N/A</td>
</tr>
<tr class="table__row">
<td class="table__cell w75">P/E Ratio (w/o extraordinary items)</td>
<td class="table__cell w25">20.00</td>
</tr>
<tr class="table__row">
<td class="table__cell w75">Price to Sales Ratio</td>
<td class="table__cell w25">1.55</td>
</tr>
<tr class="table__row">
<td class="table__cell w75">Price to Book Ratio</td>
<td class="table__cell w25">0.75</td>
</tr>
<tr class="table__row">
<td class="table__cell w75">Price to Cash Flow Ratio</td>
<td class="table__cell w25">10.90</td>
</tr>
<tr class="table__row">
<td class

In [152]:
all_data

[{'name': '\nP/E Current\n20.74\n', 'value': 'P/E Current'},
 {'name': '\nRevenue/Employee\n$675,054\n', 'value': 'Revenue/Employee'},
 {'name': '\nCurrent Ratio\n1.03\n', 'value': 'Current Ratio'},
 {'name': '\nGross Margin\n32.55%\n', 'value': 'Gross Margin'},
 {'name': '\nTotal Debt to Total Equity\n57.81\n',
  'value': 'Total Debt to Total Equity'}]